In [1]:
import pandas as pd
import numpy as np
import re

# Gensim
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)



import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# Load Data
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [4]:
#  lemmatize dan stem preprocessing 
stemmer = SnowballStemmer(language='english',ignore_stopwords=True)
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [5]:
# Preprocess  headline text, lalu simpan ke‘processed_docs’
processed_docs = documents['headline_text'].map(preprocess)
processed_docs.head(10)

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [6]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [7]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [8]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3570, 1), (3571, 1)]

In [9]:
# Preview Bag Of Words 
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3570 ("compulsori") appears 1 time.
Word 3571 ("ratepay") appears 1 time.


In [10]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    print(doc)
    break

[(0, 0.5842699484464488), (1, 0.38798859072167835), (2, 0.5008422243250992), (3, 0.5071987254965034)]


In [11]:
# proses training
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [12]:
# Menampilkan Tiap topic
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"death" + 0.025*"charg" + 0.025*"case" + 0.025*"court" + 0.021*"murder" + 0.020*"polic" + 0.015*"alleg" + 0.013*"trial" + 0.012*"arrest" + 0.012*"face"
Topic: 1 
Words: 0.022*"news" + 0.020*"market" + 0.017*"world" + 0.017*"women" + 0.015*"final" + 0.015*"australian" + 0.014*"island" + 0.012*"return" + 0.011*"street" + 0.010*"fall"
Topic: 2 
Words: 0.051*"coronavirus" + 0.029*"covid" + 0.024*"live" + 0.021*"nation" + 0.021*"coast" + 0.016*"restrict" + 0.014*"water" + 0.013*"gold" + 0.011*"plan" + 0.010*"park"
Topic: 3 
Words: 0.038*"sydney" + 0.025*"polic" + 0.021*"crash" + 0.020*"adelaid" + 0.019*"die" + 0.015*"miss" + 0.012*"break" + 0.011*"drug" + 0.011*"driver" + 0.010*"shoot"
Topic: 4 
Words: 0.037*"year" + 0.032*"melbourn" + 0.022*"open" + 0.021*"canberra" + 0.017*"jail" + 0.015*"work" + 0.014*"high" + 0.014*"life" + 0.013*"interview" + 0.013*"offic"
Topic: 5 
Words: 0.028*"govern" + 0.019*"health" + 0.019*"school" + 0.017*"help" + 0.016*"chang" + 0.015*"co